## Normalized Region --> NYC (Final Map)

#### NB setup

In [1]:
import pandas as pd
import json
import geopandas as gpd
import plotly.express as px
import os

In [3]:
# make display wider
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:88% !important; }</style>"))

# allow max rows and colums to be displayed
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# set directory explicitly
os.chdir(r'C:\Users\steve\GitHub\rp-covid-migration')

In [ ]:
# read in GeoJSON, convert to geopandas
subplaces = r'data/shapefiles/nyc_subplace_centroid.geojson'
subplaces = gpd.read_file(subplaces)
subplaces.head()

#### Read in the returns file, clean up the data for viz

In [ ]:
df = pd.read_csv(r'data/subplxborough/subxnyc_01-10-2020.csv')

decimals = 0 # to round to whole num in lambda func below


df = df.rename(columns = {'orig_subpl_str':'id'})
df['id'] = df['id'].str[1:] # remove the 'U' in the column

list_date = df.columns.to_list()
list_date = list_date[2:]
# melt so we have boroughs as a column.
df = pd.melt(df, id_vars=['id','date_y-m-d'], value_vars = list_date)
# rename the columns for clarity
df = df.rename(columns = {'variable':'Borough','date_y-m-d':'date'})
# add explicit borough column -- New York City, Bronx, Brooklyn, Manhattan, Queens, Staten Island
# borough_name = {'New York City':'NYC', 'Bronx':'BX', 'Brooklyn':'BK', 'Manhattan':'MN', 'Queens':'QN', 'Staten Island':'SI'}
df['Borough'] = df['Borough'].replace({'NYC':'New York City', 'BX':'Bronx', 'BK':'Brooklyn', 'MN':'Manhattan', 'QN':'Queens', 'SI':'Staten Island'})
# parse month out of dates
new = df["date"].str.split("/", n = 1, expand = True) 
df["month"]= new[0].astype(int) # convert to integer
# remove october dates --- for now, remove this step when you add entire year
df = df.drop(df[df.month > 9].index)
# convert to date
df['date'] = pd.to_datetime(df['date'])
# merge with counties
df = pd.merge(subplaces, df, left_on='GEOID', right_on='id', how = "inner")
# set to this date format
df['date'] = df["date"].dt.strftime('%m/%d/%Y')
# conversions for visualization purposes
df['value'] = df['value'].astype(float)
df['value'] = df['value'].apply(lambda x: round(x, decimals)) # round to nearest whole num
df['value'] = df['value'].astype(int) # remove decimal
#remove rows less that are 0, we don't want them to show.
df = df.loc[df['value']>=1]
df = df.rename(columns = {'NAME':'Name','date':'Date','value':'Returns'})
# sort for visualization
df = df.sort_values(by = 'Date')
# remove entire city
df = df[df['Borough'] != 'New York City']
# save this cleaned file
# df.to_csv(r'data/subplxborough/norm_subplace_x_nyc.csv', index = False)

df.head()

#### Plotly Visualization

In [8]:
df = pd.read_csv(r'data/subplxborough/norm_subplace_x_nyc.csv')
df = df.sort_values(by = 'Borough', ascending = True)

In [9]:
fig = px.scatter_mapbox(df,# geojson = subplaces, this geojson is not needed if you have lat longs as separate columns in df
                     lat = 'INTPTLAT', lon = 'INTPTLON',
                     size = 'Returns', # column to reference bubble sizes
                     color ='Borough', # change color based on borough
                     color_discrete_sequence = ['#CCCCFF','#1D819A','#63BB69','#FFE066','#DE7A3F'], # calling in explicit borough colors we created
                     animation_frame = 'Date',
                     animation_group = 'Borough',
                     hover_data = {'Name':True,'Date':True,'Returns':True, 'INTPTLAT':False, 'INTPTLON':False, 'Borough':False}, # specifying hover data layers
                     #font = dict(family = 'Arial', color = 'white'),
                     size_max = 60)
fig.update_layout(mapbox_zoom = 6.8,
                 mapbox_center = {'lat':40.82, 'lon':-74.25},
                 mapbox_style="carto-darkmatter",
                # size of the viz
                 width = 950,
                 height = 780, 
                 font = dict(family = 'Arial', color = 'black'),
                 title_font_color = 'white',
                # set legend params
                 legend=dict(
                      title = 'Destination',
                      title_font_color = 'white',
                      bgcolor = 'black',
                      bordercolor = 'gray', borderwidth = 1,
                      font = dict(family = 'Arial', color = 'white'),
                      yanchor="top",
                      y=0.33,
                      xanchor="left",
                      x=0.02))
## THIS IS TO BE ADDED IF STANDALONE VIZ, we are placing in HTML, this is not necessary

# TITLE
# fig.add_annotation(text = "Non-NYC Residents' Trips to NYC<br>by Home Municipality",
#                   align = 'left', x = 0.01, y = 0.95, showarrow = False,
#                   bordercolor = None,
#                   bgcolor = None,
#                   font = dict(family = "Arial", color = 'white', size = 19))

# SUBITILE
# fig.add_annotation(text = 'January 1, 2020 to October 31, 2020',
#                   align = 'left', x = 0.01, y = 0.86, showarrow = False,
#                   bordercolor = None,
#                   bgcolor = None,
#                   font = dict(family = "Arial", color = 'white', size = 13))

# EXPLANATORY TEXT
fig.add_annotation(text = 'Bubble size = # of Trips',
                  align = 'right', x = 0.02, y = 0.01, showarrow=False,
                  bordercolor = None, bgcolor = None,
                  font = dict(family = 'Droid Sans', color = 'white', size = 12))
fig.add_annotation(text = 'Click on/off boroughs in legend to compare or isolate migratory patterns',
                  align = 'right', x = 0.02, y = 0.082, showarrow=False,
                  bordercolor = None, bgcolor = None,
                  font = dict(family = 'Droid Sans', color = 'white', size = 12))
# speed this viz up so it doesn't look like crap..
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 80 
# save the HTML, send it on over
fig.write_html(r'Maps/html_maps/subplace_to_nyc.html',auto_open = True,auto_play = False, include_plotlyjs = 'cdn')